# Finetune the whisper to realize the any2chinese task

Whisper模型是OpenAI推出的语音识别模型，该模型支持多种语言的语音识别，将其转录为对应语言文本。同时，该模型还有翻译功能，能将多种语言的语音转录为英文文本。在Whisper模型诞生之后，很多开源项目对其进行了微调或改进，使得模型能够在小语种上语音识别能力得到了进一步的增强，并且在C语言编译、PEFT等技术的加持下，Whisper模型现在可以在小设备上加速运行。

但是，当前，还鲜有看到有项目进一步的开发Whisper对新语种语音转录功能，或者进一步微调模型使其支持将各种语言的语音直接转录为中文文本。显然，这将进一步的发掘模型的潜力。

本教程借助transformers包收录的Whisper模型，在其基础上对该模型的tokenizer进行了修改。这样使得Whisper模型具备转录新语种以及直接转录为中文文本的能力。本教程将该方法命名为Whisper-Any2Chinese模型。

In [1]:
from huggingface_hub import notebook_login, login
from datasets import load_dataset, DatasetDict
from pprint import pprint
from transformers import WhisperFeatureExtractor
import os
from transformers import WhisperTokenizer
from transformers import WhisperProcessor

from datasets import Audio
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

import evaluate
from transformers import WhisperForConditionalGeneration,Seq2SeqTrainingArguments
from peft import prepare_model_for_int8_training,LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model,PeftModel, PeftConfig
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl,WhisperForConditionalGeneration, Seq2SeqTrainer
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR


import gc
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/towardspring/opt/anaconda3/envs/whisper/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
login()

在微调模型之前，你需要准备好训练数据。训练数据的格式以及数据的元信息文件如本教程的“./data/*”中的两个数据集所示。其中“en_mini”表示英文数据集（包含对应的转录中文），“ug_mini”表示维语数据集（包含对应的转录中文）。你可以根据自己的需求，准备好自己的数据集。本教程的数据来自于commonvoice数据集，你可以在[这里](https://commonvoice.mozilla.org/)下载到该数据集。（只做demo，因此本教程的数据集只包含了很少的数据）

In [3]:
# load dataset
common_voice = DatasetDict()

dataset_name_or_path = "./data/ug_mini"
language = "uyghur"
language_abbr = "ug" # Short hand code for the language we want to fine-tune

common_voice["train"] = load_dataset(dataset_name_or_path, split="train")
common_voice["test"] = load_dataset(dataset_name_or_path, split="test")
common_voice["validation"] = load_dataset(dataset_name_or_path, split="validation")

print(common_voice)

Resolving data files:   0%|          | 0/55 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/55 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/55 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence', 'chinese'],
        num_rows: 55
    })
    test: Dataset({
        features: ['audio', 'sentence', 'chinese'],
        num_rows: 12
    })
    validation: Dataset({
        features: ['audio', 'sentence', 'chinese'],
        num_rows: 15
    })
})


In [5]:
# load model
model_name_or_path = "openai/whisper-small"   # you can choose any size of model by your service's capability
task = "translate"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language=language_abbr, task=task)

Whisper本身是不支持添加新语种或者多语言中文转录功能。因此，我们需要在微调之前，添加新的语种标签以及中文转录翻译提示标签。例如，若想实现对维吾尔语的转录，我们需要向tokenizer模块添加<|ug|>标签；若想实现对多语种的中文转录翻译功能，需要向tokenizer模块添加<|translate-chinese|>标签。本教材只对英语语音转录翻译中文文本做出示范，您可以根据您的需要添加任意语音或翻译标签。